In [1]:
import re
import email
import os
from stop_words import get_stop_words
from gensim import corpora, models, utils
from nltk.stem.porter import PorterStemmer
from nltk import clean_html
from bs4 import BeautifulSoup
from collections import defaultdict

import pyLDAvis
import pyLDAvis.gensim

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pwd

'C:\\Users\\EricJulienASKINAZI\\Documents\\mail\\email_classification\\notebooks'

In [3]:
# Read emails from csv

file_name = os.path.join('../data/email_perso.csv')
full_data = pd.read_csv(file_name, encoding='latin1')

full_data = full_data.loc[[5, 6]]
full_data.loc[5, 'Body'] = "a b c"
full_data.loc[6, 'Body'] = "a b c d e f"


print(type(full_data))

body = full_data["Body"]
subjects = full_data["Subject"]
#print(full_data)

<class 'pandas.core.frame.DataFrame'>


In [4]:
# On modifie le dataframe panda pour standardiser les titres
full_data['Subject'] =  [re.sub("((re)|(fwd?)).?: ", "", str(x).lower()) for x in full_data['Subject']]

#display(full_data)
display(full_data[['Date Sent', 'Subject', 'Body', 'From (address)', 'To (address)']][:5])

,Date Sent,Subject,Body,From (address),To (address)
5,14/05/2019 16:04,scb agrees to award us the project,a b c,/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,/o=ExchangeLabs/ou=Exchange Administrative Gro...
6,14/05/2019 16:19,scb agrees to award us the project,a b c d e f,/O=EXCHANGELABS/OU=EXCHANGE ADMINISTRATIVE GRO...,/o=ExchangeLabs/ou=Exchange Administrative Gro...


In [5]:
# For each mail we fetch all the mails with the same subject, we take the most recent mail as a basis and we add
# the content of the older mails in its body if it's not already contained in the mail

# To remove an anwanted warning
pd.options.mode.chained_assignment = None  

# Removes caps and "Re","Fwd"
full_data['Subject'] =  [re.sub("((re)|(fwd?)).?: ", "", str(x).lower()) for x in full_data['Subject']]


to_drop = []
to_keep = []

for index, row in full_data.iterrows():

    if(index not in to_drop and index not in to_keep) :
        same_subject_df = full_data.loc[full_data['Subject'] == row['Subject']]

        if len(same_subject_df) > 1 :
            most_recent_index = same_subject_df.index[-1]
            to_keep.append(most_recent_index)
            
            most_recent = same_subject_df.loc[most_recent_index]

            for s_index, s_row in same_subject_df.iterrows() :
                if s_index != most_recent_index :
                    if s_row["Body"] not in most_recent["Body"] :

                        most_recent["Body"] += " " + s_row["Body"]

                    to_drop.append(s_index)
                    
        else :
            to_keep.append(index)

unique_data = full_data.drop(to_drop)

    
    
    

In [ ]:
unique_data.style

In [ ]:
def word_frequency_filter(texts, count_threshold):
    """Remove word whose frequency is less than a count threshold
    """

    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    return [[token for token in text if frequency[token] > count_threshold] for text in texts ] 

In [ ]:
# Text preprocess
processed_msg = {}
raw_message = []
for key, document in cleaned_msg.items():
    # raw = document.lower() # use less with simple token
    tokens = utils.simple_preprocess(document, min_len=3)
    # print(tokens)
    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if (not i in en_stop)]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    print(key)
    # Lemmatize ?
    print(stemmed_tokens)
    # Remove tokens that not appear enough in the corpus
    # remove words that appear only once
    st_tokens = word_frequency_filter(stemmed_tokens, 2)
    print("-"*100)
    
    processed_msg[key] = stemmed_tokens
    
    
    raw_message.append(document)


In [ ]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary([ email for _, email in processed_msg.items()])

In [ ]:
number_topic = 7
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for _, text in processed_msg.items()]

# generate LDA model
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=number_topic, id2word=dictionary, passes=100)

ldamodel.print_topics(num_topics = number_topic, num_words=10)

In [ ]:
for count, c in enumerate(ldamodel[corpus]):
    print(c)
    print("document number : {}".format(count))   
    print("------------------------------------------------------\n")

In [ ]:
for count, c in enumerate(ldamodel[corpus]):
    print("document number : {}".format(count))
    print(cleaned_msg[key_list[count]])
    print("Results     : ", c)
    print("------------------------------------------------------\n")

In [ ]:
vis = pyLDAvis.gensim.prepare(topic_model=ldamodel, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [ ]:
ldamodel.get_document_topics(corpus[0])

In [ ]:
ldamodel.get_topics()

In [ ]:
ldamodel.show_topics()

In [ ]:
topic = 4
for count, c in enumerate(ldamodel[corpus]):
    for cc in c:
        if cc[0] == topic:
            print("document number : {}".format(count))
            print(cleaned_msg[key_list[count]])
            print("\n")
            print("Topics number      : ", c[0])
            print(ldamodel.print_topic(c[0][0]))
            print("\n")
            print("similarity index : {}".format(c[0][1]))
            print("\n")
            print(processed_msg[count])
            print("------------------------------------------------------\n")

In [ ]:
msg_len = [len(mess) for _, mess in cleaned_msg.items()]
plt.hist(msg_len, 100)

Minimal length of documet should be 70 character

In [ ]:
"Hello Christophe & Hussam Thanks to your hard work and teamwork , I am pleased to say that Upskills is going to be a vendor with scb. I just had a coffee with Saim and he just gave the green signal for on-boarding process. Well done and congrats on your achievement . Thank you Regards George Get Outlook for iOS" in "Hi George, Hussam and Christophe, Great job! Thank you for your effort. Van Anh From: George VARGHESE Sent: Tuesday, 14 May 2019 4:05 PM To: Christophe BOETTO ; Hussam HAMDAN ; Aurelien BAELDE Cc: Thong TRAN ; Van Anh DO Subject: Scb agrees to award us the project Hello Christophe & Hussam Thanks to your hard work and teamwork , I am pleased to say that Upskills is going to be a vendor with scb. I just had a coffee with Saim and he just gave the green signal for on-boarding process. Well done and congrats on your achievement . Thank you Regards George Get Outlook for iOS"